In [3]:
# -*- coding: utf-8 -*-
"""
Converted from VBASim by
Yujing Lin, Linda Pei & Barry L Nelson
Last update 8/15/2018
"""


import SimClasses
import SimFunctions
import SimRNG
import math
import pandas
import numpy as np
 
SimClasses.Clock = 0
MeanParkingTime = 2.0
QueueLength = SimClasses.CTStat()
N = 0
MaxQueue = 0
RepNum = 300

ZSimRNG = SimRNG.InitializeRNSeed()
Calendar = SimClasses.EventCalendar()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheCTStats.append(QueueLength)

AllQueueLength = []
AllMaxQueue = []
AllN = []

print("Average Number in Queue", "Maximum Number in Queue")
 
def NSPP(Stream):
    PossibleArrival = SimClasses.Clock + SimRNG.Expon(1.0/110, Stream)
    while SimRNG.Uniform(0, 1, Stream) >= (100 + 10 * math.sin(3.141593 * PossibleArrival / 12)) / 110.0:
        PossibleArrival = PossibleArrival + SimRNG.Expon(1.0/110, Stream)
    nspp = PossibleArrival - SimClasses.Clock
    return nspp

def Arrival():
    global MaxQueue
    global N
    interarrival = NSPP(1)
    SimFunctions.Schedule(Calendar,"Arrival", interarrival)
    N = N + 1
    QueueLength.Record(N)
    if N > MaxQueue:
        MaxQueue = N   
    SimFunctions.Schedule(Calendar,"Departure",SimRNG.Expon(MeanParkingTime, 2))

def Departure():
    global N
    N = N - 1
    QueueLength.Record(N)
    
for Reps in range(0,RepNum,1):
    N = 0
    MaxQueue = 0
    SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
    interarrival = NSPP(1)
    SimFunctions.Schedule(Calendar,"Arrival",interarrival)
    SimFunctions.Schedule(Calendar,"EndSimulation", 24)
    
    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Arrival":
        Arrival()
    elif NextEvent.EventType == "Departure":
        Departure()
    
    while NextEvent.EventType != "EndSimulation":
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Arrival":
            Arrival()
        elif NextEvent.EventType == "Departure":
            Departure()
    
    AllQueueLength.append(QueueLength.Mean())
    AllMaxQueue.append(MaxQueue)
    AllN.append(N) 
    
    print(Reps+1, QueueLength.Mean(), MaxQueue, N)

# estimating the 0.99th quantile
quantile_index = int(np.ceil(RepNum*0.99)-1)
print("Estimated required capacity is:", np.sort(AllMaxQueue)[quantile_index])
print("Estimated Expected Average # of cars:", np.mean(AllQueueLength))
print("Estimated Expected Max # of cars:", np.mean(AllMaxQueue))

Average Number in Queue Maximum Number in Queue
1 184.62726696060008 224 173
2 184.28249425329898 246 153
3 181.04534534643392 243 189
4 180.87055134440493 231 190
5 189.3270891400119 233 189
6 188.17191132085273 231 204
7 185.23932728957604 239 193
8 184.60077678012192 236 184
9 193.1353663463818 253 208
10 180.7623906316907 229 160
11 181.98699751615877 239 177
12 178.64557034254184 222 202
13 180.937852931256 239 203
14 181.28993043360438 222 220
15 186.75408312124196 229 191
16 192.61598663991344 232 193
17 182.06717696341494 231 208
18 184.7505425007724 232 201
19 183.40031376502245 243 189
20 189.76328305242995 234 190
21 184.63191042087445 223 200
22 193.13518760302188 244 203
23 186.98130422428423 265 203
24 185.84422663211146 238 213
25 185.64466155417395 225 192
26 175.13262391561668 242 188
27 185.77949719056468 245 192
28 187.91477129395398 257 191
29 178.9319777225413 249 162
30 181.8128147881761 230 205
31 178.80469898089552 227 173
32 179.15709186956647 218 206
33 186.57

In [4]:
# Expected average number of cars in the parking lot during a day
E_mean = np.mean(AllQueueLength)

# Expected max number of cars in the parking lot during a day
E_max = np.mean(AllMaxQueue)

# Probability having more than 5 cars left in parking lot
more_than_5_cars_left = (np.array(AllN) > 5).astype(int)
prob_more_than_5 = sum(more_than_5_cars_left) / len(more_than_5_cars_left)

# 0.95 quantile max number of cars
quantile = np.sort(AllMaxQueue)[int(len(AllMaxQueue) * 0.95) + 1]


E_mean, E_max, prob_more_than_5, 258

(184.26854846599448, 236.88666666666666, 1.0, 258)

# Q3

In [1]:
import matplotlib.pyplot as plt
import numpy as np
def CI_95(data):
    a = np.array(data)
    n = len(a)
    m = np.mean(a)
    sd = np.std(a,ddof=1)
    hw = 1.96*sd / np.sqrt(n)
    return m, [m-hw,m+hw]

def down_out_option(B):
    B = 35
    Maturity = 1.0
    InterestRate = 0.05
    Sigma = 0.3
    InitialValue = 50.0
    StrikePrice = 55.0
    Steps = 64
    Interval = Maturity / Steps
    Sigma2 = Sigma * Sigma / 2
    np.random.seed(1)
    Replications = 40000
    Interval = Maturity / Steps
    ValueList = [] # List to keep the option value for each sample path

    I = 0

    for i in range(0,Replications):
        Sum = 0.0
        X = InitialValue
        for j in range(0,Steps):
            Z = np.random.standard_normal(1)

            X = X * np.exp((InterestRate - Sigma2) * Interval +
                           Sigma * np.sqrt(Interval) * Z)

            if X < B:
                I = 1

            Sum = Sum + X
        Value = np.exp(-InterestRate * Maturity) * max(Sum/Steps - StrikePrice, 0) * I
        ValueList.append(Value)  

    print ("Mean and CI:", CI_95(ValueList))

In [2]:
down_out_option(35)

Mean and CI: (array([2.0841373]), [array([2.03937869]), array([2.12889591])])


/tmp/ipykernel_108487/2730985832.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.array(data)


In [ ]:
down_out_option(40)

In [ ]:
down_out_option(45)

# Q4

In [1]:
import SimFunctions
import SimRNG 
import SimClasses
import numpy as np
import pandas


ZSimRNG = SimRNG.InitializeRNSeed()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Server = SimClasses.Resource()
Calendar = SimClasses.EventCalendar()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheDTStats.append(Wait)
TheQueues.append(Queue)
TheResources.append(Server)

s = 2
Server.SetUnits(s) 
MeanTBA = 1.0
MeanST = 0.8 * s
Phases = 3
RunLength = 550.0
WarmUp = 50.0

AllWaitMean = []
AllQueueMean = []
AllQueueNum = []
AllServerMean = []
print ("Rep", "Average Wait", "Average Number in Queue", "Number Remaining in Queue", "Server Utilization")

def Arrival():
    SimFunctions.Schedule(Calendar,"Arrival",SimRNG.Expon(MeanTBA, 1))
    Customer = SimClasses.Entity()
    Queue.Add(Customer)
    
    if (Server.Busy < s) and (Server.Busy >= 0):
        seized = Server.Seize(1)
        SimFunctions.Schedule(Calendar,"EndOfService",SimRNG.Erlang(Phases,MeanST,2))
        print(f" server has {Server.Busy} busy units and we decided {seized}-schedule")
    else:
        print(f" server has {Server.Busy} busy units and we decided not to-schedule service")

def EndOfService():
    DepartingCustomer = Queue.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
    if Queue.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService",SimRNG.Erlang(Phases,MeanST,2))
    else:
        Server.Free(1)

for reps in range(0,10,1):

    SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
    SimFunctions.Schedule(Calendar,"Arrival",SimRNG.Expon(MeanTBA, 1))
    SimFunctions.Schedule(Calendar,"EndSimulation",RunLength)
    SimFunctions.Schedule(Calendar,"ClearIt",WarmUp)
    
    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Arrival":
        Arrival()
    elif NextEvent.EventType == "EndOfService":
        EndOfService() 
    elif NextEvent.EventType == "ClearIt":
        SimFunctions.ClearStats(TheCTStats,TheDTStats)
    
    while NextEvent.EventType != "EndSimulation":
        print(NextEvent.EventType)
        print("people in queue", Queue.NumQueue())
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Arrival":
            Arrival()
        elif NextEvent.EventType == "EndOfService":
            EndOfService()
        elif NextEvent.EventType == "ClearIt":
            SimFunctions.ClearStats(TheCTStats,TheDTStats)
    
    
    AllWaitMean.append(Wait.Mean())
    AllQueueMean.append(Queue.Mean())
    AllQueueNum.append(Queue.NumQueue())
    AllServerMean.append(Server.Mean())
    print (reps+1, Wait.Mean(), Queue.Mean(), Queue.NumQueue(), Server.Mean())

# output results

print("Estimated Expected Average wait:",np.mean(AllWaitMean))
print("Estimated Expected Average queue-length:", np.mean(AllQueueMean))
print("Estimated Expected Average utilization:",np.mean(AllServerMean))

# convert the output to a dataframe and write to a .csv file
output = {"AllWaitMean" : AllWaitMean, "AllQueueMean": AllQueueMean, "AllQueueNum" : AllQueueNum, "AllServerMean": AllServerMean}
output = pandas.DataFrame(output)
output.to_csv("MG1_output.csv", sep=",")



Rep Average Wait Average Number in Queue Number Remaining in Queue Server Utilization
 server has 1.0 busy units and we decided True-schedule
Arrival
people in queue 1
EndOfService
people in queue 0
 server has 1.0 busy units and we decided True-schedule
Arrival
people in queue 1
 server has 2.0 busy units and we decided True-schedule
Arrival
people in queue 2
 server has 2.0 busy units and we decided not to-schedule service
Arrival
people in queue 3
EndOfService
people in queue 2
 server has 2.0 busy units and we decided not to-schedule service
Arrival
people in queue 3
EndOfService
people in queue 2
 server has 2.0 busy units and we decided not to-schedule service
Arrival
people in queue 3
EndOfService
people in queue 2
 server has 2.0 busy units and we decided not to-schedule service
Arrival
people in queue 3
EndOfService
people in queue 2
 server has 2.0 busy units and we decided not to-schedule service
Arrival
people in queue 3
EndOfService
people in queue 2
EndOfService
people in

AttributeError: 'NoneType' object has no attribute 'CreateTime'

In [3]:
"""
Converted from VBASim by
Yujing Lin, Linda Pei & Barry L Nelson
Last update 8/15/2018
"""

import SimFunctions
import SimRNG 
import SimClasses
import numpy as np
import pandas


ZSimRNG = SimRNG.InitializeRNSeed()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Server = SimClasses.Resource()
Calendar = SimClasses.EventCalendar()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheDTStats.append(Wait)
TheQueues.append(Queue)
TheResources.append(Server)

Server.SetUnits (1) 
MeanTBA = 1.0
MeanST = 0.8
MeanTR = 1.0
c = 4
Phases = 3
RunLength = 55000.0
WarmUp = 5000.0

AllWaitMean = []
AllQueueMean = []
AllQueueNum = []
AllServerMean = []
print ("Rep", "Average Wait", "Average Number in Queue", "Number Remaining in Queue", "Server Utilization")

def Arrival():
    SimFunctions.Schedule(Calendar,"Arrival",SimRNG.Expon(MeanTBA, 1))
    Customer = SimClasses.Entity()
    
    if Queue.NumQueue() + 1 >= c:
        SimFunctions.Schedule(Calendar,"Retrial",SimRNG.Expon(MeanTR,3))
        
        
    Queue.Add(Customer)
    
    if Server.Busy == 0:
        Server.Seize(1)
        SimFunctions.Schedule(Calendar,"EndOfService",SimRNG.Erlang(Phases,MeanST,2))

def EndOfService():
    DepartingCustomer = Queue.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
    if Queue.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService",SimRNG.Erlang(Phases,MeanST,2))
    else:
        Server.Free(1)

def Retrial():
    Customer = SimClasses.Entity()
    Queue.Add(Customer)
        
for reps in range(0,10,1):

    SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
    SimFunctions.Schedule(Calendar,"Arrival",SimRNG.Expon(MeanTBA, 1))
    SimFunctions.Schedule(Calendar,"EndSimulation",RunLength)
    SimFunctions.Schedule(Calendar,"ClearIt",WarmUp)
    
    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Arrival":
        Arrival()
    elif NextEvent.EventType == "EndOfService":
        EndOfService() 
    elif NextEvent.EventType == "ClearIt":
        SimFunctions.ClearStats(TheCTStats,TheDTStats)
    
    while NextEvent.EventType != "EndSimulation":
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Arrival":
            Arrival()
        elif NextEvent.EventType == "EndOfService":
            EndOfService()
        elif NextEvent.EventType == "Retrial":
            Retrial()
        elif NextEvent.EventType == "ClearIt":
            SimFunctions.ClearStats(TheCTStats,TheDTStats)
    
    AllWaitMean.append(Wait.Mean())
    AllQueueMean.append(Queue.Mean())
    AllQueueNum.append(Queue.NumQueue())
    AllServerMean.append(Server.Mean())
    print (reps+1, Wait.Mean(), Queue.Mean(), Queue.NumQueue(), Server.Mean())

# output results

print("Estimated Expected Average wait:",np.mean(AllWaitMean))
print("Estimated Expected Average queue-length:", np.mean(AllQueueMean))
print("Estimated Expected Average utilization:",np.mean(AllServerMean))

# convert the output to a dataframe and write to a .csv file
output = {"AllWaitMean" : AllWaitMean, "AllQueueMean": AllQueueMean, "AllQueueNum" : AllQueueNum, "AllServerMean": AllServerMean}
output = pandas.DataFrame(output)
output.to_csv("MG1_output.csv", sep=",")



NameError: name 'UnitsServer' is not defined

Nursing

In [102]:
import SimFunctions
import SimRNG 
import SimClasses
import numpy as np
import pandas


ZSimRNG = SimRNG.InitializeRNSeed()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Server = SimClasses.Resource()
Calendar = SimClasses.EventCalendar()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheDTStats.append(Wait)
TheQueues.append(Queue)
TheResources.append(Server)

Server.SetUnits (1) 
MeanTBA = 1.0
MeanST = 0.8
Phases = 3
RunLength = 55000.0
WarmUp = 5000.0
staff = 170

AllWaitMean = []
AllQueueMean = []
AllQueueNum = []
AllServerMean = []
print ("Rep", "Average Wait", "Average Number in Queue", "Number Remaining in Queue", "Server Utilization")

def Arrival():
    Customer = SimClasses.Entity()
    Queue.Add(Customer)
    
    if Server.Busy == 0:
        Server.Seize(1)
        SimFunctions.Schedule(Calendar,"EndOfService",SimRNG.Erlang(Phases,MeanST,2))

def EndOfService():
    DepartingCustomer = Queue.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
    if Queue.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService",SimRNG.Erlang(Phases,MeanST,2))
    else:
        Server.Free(1)

for reps in range(0,10,1):

    SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
    
    for i in range(staff):
        SimFunctions.Schedule(Calendar,"Arrival",SimRNG.Uniform(MeanTBA, 1))
    
    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Arrival":
        Arrival()
    elif NextEvent.EventType == "EndOfService":
        EndOfService() 
    elif NextEvent.EventType == "ClearIt":
        SimFunctions.ClearStats(TheCTStats,TheDTStats)
    
    while NextEvent.EventType != "EndSimulation":
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Arrival":
            Arrival()
        elif NextEvent.EventType == "EndOfService":
            EndOfService()
        elif NextEvent.EventType == "ClearIt":
            SimFunctions.ClearStats(TheCTStats,TheDTStats)
    
    AllWaitMean.append(Wait.Mean())
    AllQueueMean.append(Queue.Mean())
    AllQueueNum.append(Queue.NumQueue())
    AllServerMean.append(Server.Mean())
    print (reps+1, Wait.Mean(), Queue.Mean(), Queue.NumQueue(), Server.Mean())

# output results

print("Estimated Expected Average wait:",np.mean(AllWaitMean))
print("Estimated Expected Average queue-length:", np.mean(AllQueueMean))
print("Estimated Expected Average utilization:",np.mean(AllServerMean))

# convert the output to a dataframe and write to a .csv file
output = {"AllWaitMean" : AllWaitMean, "AllQueueMean": AllQueueMean, "AllQueueNum" : AllQueueNum, "AllServerMean": AllServerMean}
output = pandas.DataFrame(output)
output.to_csv("MG1_output.csv", sep=",")



Rep Average Wait Average Number in Queue Number Remaining in Queue Server Utilization
1 3.056138296024382 3.0510244911889055 1 0.8031095083100359
2 3.0917399417823495 3.102771415466134 18 0.8029359115064603
3 2.8829115466153605 2.874895217241247 0 0.7958283399006371
4 2.9481839150959854 2.951415198438999 0 0.7990437513843559


KeyboardInterrupt: 